Upgrading numpy first. pygrib requires up to date numpy while colab has an old version of numpy, perhaps for compatibility with tensorflow. If the upgrade was needed, the runtime kills itself and restarts, then you need to run the notebook again. 

In [1]:
import numpy as np, os
if not [int(i) for i in np.__version__.split('.')] >= [1,20,1]: # check numpy version
  print('Upgrading numpy and stopping RUNTIME! When the notebook completes, please run again.')
  ! pip install --upgrade numpy    # suggested by Efosa, see also https://github.com/jswhit/pygrib/issues/192
  os.kill(os.getpid(), 9)          # kill the runtime, need to run again from the beginning

Installing dependences:

In [2]:
! pip install pygrib   
! wget --no-clobber https://raw.githubusercontent.com/openwfm/wrfxpy/master/src/ingest/grib_file.py
from grib_file import GribFile     # Martin's utility layer on top of  pygrib,from wrfxpy
import numpy as np, os # imported before, just in case


File ‘grib_file.py’ already there; not retrieving.



Create a function to transfer RTMA file in GRIP2 format from the stash. The function returns zero if the file transfer succeeded. If the file is not available, it returns a nonzero value. Note: if needed, maybe in future add more sophisticated checks, check the return code of wget and if the file size is correct.

In [3]:
import subprocess,os
def load_rtma(path,file):
  url='http://math.ucdenver.edu/~jmandel/rtma/' + path
  if os.path.exists(file):
    print(file + ' already exists, removing')
    os.remove(file)
  try:
    ret = subprocess.check_output(['wget','--no-clobber','--output-document='+ file, url,],stderr=subprocess.STDOUT).decode() # execute command from python strings
    if os.path.exists(file):
      print('loaded ' + url + ' as ' + file)
      return 0
    else: 
      print('file transfer completed, but the file is missing? ' + url)  
      return 1
  except:
    print('file transfer failed: ' + url)
    return 2


Testing the file transfer:

In [4]:
year, month, day, hour, variable  = (2018, 5, 18, 00, 'temp')  # year, month, day, hour, variable
file = variable + '.grib'
path = '%4i%02i%02i/%02i/%s' % (year, month, day, hour, file)

ret=load_rtma(path,file)   # load the file; ret is the return value that can be tested in application



loaded http://math.ucdenver.edu/~jmandel/rtma/20180518/00/temp.grib as temp.grib


Trying to read the file and a basic sanity check if the values make sense:

In [5]:
gf = GribFile(file)[1]        # grib file consists of "messages" (basically, variables), rtma files have only one
lats, lons = gf.latlons()     # grid of geo coodinates (computed), should be the same for all rtma files here
lats = np.array(lats)         # tuple to numpy array
lons = np.array(lons)         # tuple to numpy array
temp = np.array(gf.values())  # the actual variable values (here, T at 2m in K)

print('min lats %s max %s' % (np.amin(lats),np.amax(lats)))
print('shape',lats.shape)
print('min lons %s max %s' % (np.amin(lons),np.amax(lons)))
print('shape',lons.shape)
print('min temp %s max %s' % (np.amin(temp),np.amax(temp)))
print('shape',temp.shape)


min lats 20.191999000000006 max 52.80766860325035
shape (1377, 2145)
min lons -130.10343806997807 max -60.885557729221034
shape (1377, 2145)
min temp 266.84000000000003 max 315.01
shape (1377, 2145)


One special grib file with the terrain height is stored at the root of the stash. This file is a part of the RTMA dataset but no need to download and store every hour, the data should never change. Trying to read it and doing a sanity check. Also,checking if the grid coordinages in this file are the same as before.


In [6]:
hf='ds.terrainh.bin'   # terrain height, same in rtma at all times
load_rtma(hf,hf)
gf = GribFile(hf)[1] 
hgt = np.array(gf.values()) # height in m
print('min height %s max %s' % (np.amin(hgt),np.amax(hgt)))
print('shape',hgt.shape)
hlats, hlons = gf.latlons()     # grid of geo coodinates (computed), should be the same for all rtma files here
hlats = np.array(hlats)         # tuple to numpy array
hlons = np.array(hlons) 
print('difference in lats %s lons %s' % (np.amax(np.absolute(lats-hlats)), np.amax(np.absolute(lons-hlons))))


ds.terrainh.bin already exists, removing
loaded http://math.ucdenver.edu/~jmandel/rtma/ds.terrainh.bin as ds.terrainh.bin
min height -76.60000000000001 max 4156.900000000001
shape (1377, 2145)
difference in lats 0.0 lons 0.0
